In [30]:
import pandas as pd
import numpy as np
import math

In [33]:
# https://www.google.com/basepages/producttype/taxonomy.en-US.txt

cat = pd.read_csv("./data/taxonomy.csv", encoding="ISO-8859-1")
cat.fillna(False, inplace=True)
print(cat.shape)
cat.head(2)

(5427, 7)


,category_1,category_2,category_3,category_4,category_5,category_6,category_7
0,Animals & Pet Supplies,False,False,False,False,False,False
1,Animals & Pet Supplies,Live Animals,False,False,False,False,False


In [9]:
df = pd.read_csv("./data/emails.csv", encoding="ISO-8859-1")
print(df.shape)
df.head(2)

(100, 2)


,subject,full_text
0,Prana Patterns on Point + a FREE $20,PLUS the $20 Almost Gift Card To view this ema...
1,Now 25% OFF every doggone thing. Everything. D...,The September Style sale The September Style s...


In [40]:
# write a fn to read an email subject and find the corresponding categories

sample = df.iloc[13].subject

category = []
for i, row in cat.iterrows():
    if row["category_4"]:
        words = row["category_4"].split(" ")
        for w in words:
            if w in sample:
                category.append(i)

In [63]:
def print_categories(list_of_ids):
    for i in list_of_ids:
        category = cat.iloc[i]
        print(" > ".join([i for i in category if i != False]))
        
print_categories(category)

Animals & Pet Supplies > Live Animals
Animals & Pet Supplies > Pet Supplies > Bird Supplies > Bird Cage Accessories > Bird Cage Bird Baths
Animals & Pet Supplies > Pet Supplies > Pet Training Aids > Pet Training Clickers & Treat Dispensers


In [ ]:
# next steps: do this for all 100 subjects
# do this for all subjects and full_text